In [20]:
import pandas as pd 

### Загружаем данные 

In [21]:
lessons = pd.read_csv('tech_quality/lessons.txt',names = ['id', 'event_id', 'subject', 'time'],
                   sep = '|', skipinitialspace=True)
lessons.drop(lessons.index[[0, 1, -1]], inplace= True)

In [22]:
lessons.head()

,id,event_id,subject,time
2,1e7bb408-cfef-4a9f-8328-351c9483a64c,38114,phys,2020-01-19 12:00:00
3,6d8e59d9-a7c8-4bb3-8ff3-99cd07acdf1a,51568,it,2020-01-19 13:00:00
4,62e1a078-33de-47c1-99d2-845b1daca56f,52790,hist,2020-01-19 13:00:00
5,00fc6685-f53a-49bb-b960-5e0042fd3852,51341,phys,2020-01-17 12:00:00
6,4cadf623-82e6-422f-a342-acf978302fb2,55048,phys,2020-01-19 14:00:00


In [23]:
quality = pd.read_csv('tech_quality/quality.txt',names = ['lesson_id', 'tech_quality'],
                   sep = '|', skipinitialspace=True)
quality.drop(quality.index[[0,1,-1]], inplace = True)

In [24]:
quality.head()

,lesson_id,tech_quality
2,6d8e59d9-a7c8-4bb3-8ff3-99cd07acdf1a,5
3,62e1a078-33de-47c1-99d2-845b1daca56f,5
4,62e1a078-33de-47c1-99d2-845b1daca56f,5
5,00fc6685-f53a-49bb-b960-5e0042fd3852,5
6,00fc6685-f53a-49bb-b960-5e0042fd3852,5


In [25]:
users = pd.read_csv('tech_quality/users.txt',names = ['id', 'role'],
                   sep = '|', skipinitialspace=True)
users.drop(users.index[[0,1,-1]], inplace = True)

In [26]:
users.head()

,id,role
2,e28351f5-4ccb-4549-8647-d43f2b15e7b8,pupil
3,4df2832a-1d63-4453-9659-43993fc35996,tutor
4,bb1c0bc8-1212-452b-97a0-439d4a2169e2,pupil
5,63441abe-c4da-4275-ba26-66f7dbd65dde,tutor
6,a1323b68-c82c-429a-8e2c-65597e648c1a,tutor


In [27]:
participants = pd.read_csv('tech_quality/participants.txt',names = ['event_id','user_id'],
                   sep = '|', skipinitialspace=True)
participants.drop(participants.index[[0,1,-1]], inplace = True)

In [28]:
participants.head()

,event_id,user_id
2,38114,e28351f5-4ccb-4549-8647-d43f2b15e7b8
3,38114,4df2832a-1d63-4453-9659-43993fc35996
4,51568,bb1c0bc8-1212-452b-97a0-439d4a2169e2
5,51568,63441abe-c4da-4275-ba26-66f7dbd65dde
6,52790,a1323b68-c82c-429a-8e2c-65597e648c1a


### Удаляем лишние пробелы

In [29]:
def cleaner(s):
    if type(s) == str:
        return s.strip(' ')

In [30]:
for col in lessons.columns:
    lessons[col] = lessons[col].apply(cleaner)

In [31]:
for col in quality.columns:
    quality[col] = quality[col].apply(cleaner)

In [32]:
for col in users.columns:
    users[col] = users[col].apply(cleaner)

In [33]:
for col in participants.columns:
    participants[col] = participants[col].apply(cleaner)

### 1.Найти все уроки по физике (subject=phys)

In [34]:
lessons[lessons['subject']=='phys']

,id,event_id,subject,time
2,1e7bb408-cfef-4a9f-8328-351c9483a64c,38114,phys,2020-01-19 12:00:00
5,00fc6685-f53a-49bb-b960-5e0042fd3852,51341,phys,2020-01-17 12:00:00
6,4cadf623-82e6-422f-a342-acf978302fb2,55048,phys,2020-01-19 14:00:00
8,e597a79a-3f68-4d15-bc77-c7000bcf8e52,53026,phys,2020-01-19 15:00:00
10,ea6f0bf3-bcfc-4555-9b16-cc2cfe722e17,51645,phys,2020-01-19 14:00:00
...,...,...,...,...
368,e90f529b-1061-4efd-9bba-f73ecee2ef27,55490,phys,2020-01-19 08:00:00
369,f0002160-e57e-4e90-ac7e-c8fcdbbae26f,41963,phys,2020-01-19 09:00:00
370,8a1bac54-c5c6-47c9-8fdb-36d01384eaae,55881,phys,2020-01-19 15:00:00
371,457accf4-bd99-4b9d-b540-ef200e65afbd,49007,phys,2020-01-19 05:00:00


### небольшая предобработка и преобразование типов для выполнения последующих пунктов

In [37]:
tutors = users[users['role'] == 'tutor']['id']

lessons['time'] = pd.to_datetime(lessons['time'])

temp=lessons.rename(columns={'id':'lesson_id'}).merge(quality, how='outer')
temp['tech_quality']=temp['tech_quality'].astype('float')

In [38]:
days = temp['time'].apply(lambda x:x.strftime('%Y-%m-%d')).unique()
day_marks=pd.DataFrame()
day_marks['time'] = days
day_marks['time'] = day_marks['time'].apply(lambda x: str(x)[:11])
day_marks.sort_values('time', inplace = True)
day_marks["time"].values

array(['2020-01-11', '2020-01-12', '2020-01-13', '2020-01-14',
       '2020-01-15', '2020-01-16', '2020-01-17', '2020-01-18',
       '2020-01-19', '2020-01-20'], dtype=object)

In [39]:
for tutor in tutors:
    tutor_events = participants[participants['user_id'] == tutor]['event_id']
    answer = temp[ 
                    (temp['event_id'].isin(tutor_events)) 
                    & (temp['subject'] == 'phys')
                 ].groupby(pd.Grouper(key = 'time', freq="D"))['tech_quality'].mean().round(2); 
    answer=answer.rename_axis('time').reset_index()
    answer['time']=answer['time'].apply(lambda x: str(x)[:10])
    answer.rename(columns={'tech_quality':tutor})
    day_marks = day_marks.merge(answer.rename(columns={'tech_quality':tutor}), how='left')

In [40]:
day_marks.index = day_marks['time']
day_marks.drop('time', axis= 1, inplace = True)

### 2. В каждый день  для каждого репетитора посчитать среднюю арифметическую оценку за его уроки (учитывать только уроки из п. 1).

In [41]:
day_marks.T

time,2020-01-11,2020-01-12,2020-01-13,2020-01-14,2020-01-15,2020-01-16,2020-01-17,2020-01-18,2020-01-19,2020-01-20
4df2832a-1d63-4453-9659-43993fc35996,NaN,5.0,NaN,5.0,NaN,5.0,5.00,5.00,5.00,NaN
63441abe-c4da-4275-ba26-66f7dbd65dde,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
a1323b68-c82c-429a-8e2c-65597e648c1a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30a19496-bdaf-461c-abbc-2709ae520201,5.0,NaN,5.0,NaN,NaN,5.0,5.00,5.00,NaN,4.5
43efce48-94b2-4412-857f-223d45969008,5.0,5.0,NaN,5.0,NaN,5.0,4.67,4.25,4.88,4.5
...,...,...,...,...,...,...,...,...,...,...
1c39e78f-6ded-4b2e-83d6-036ca34ecfdc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.00,NaN,NaN
d754610c-c4ac-46f5-9b02-8067e0f8a01d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4758da03-f56d-4cf6-a58d-17dd3d7d8445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c7123879-75e7-4957-b809-f2f684499b67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 3. Найти учителя, который в этот день имеет самую низкую среднюю арифметическую оценку за уроки (среди всех учителей, проводивших уроки по физике в этот день).

In [44]:
for col in day_marks.T.columns:
    wrost_mark = day_marks.T[col].apply(lambda x: float(x)).min()
    bad_teacher = day_marks.T[day_marks.T[col] ==wrost_mark].index[0]
    print ('{}\t{}'.format(col, bad_teacher))

2020-01-11	8fe03f08-8581-430c-a590-9888ab36deb3
2020-01-12	696c838e-c054-4e9f-a51a-50bf5660f364
2020-01-13	30a19496-bdaf-461c-abbc-2709ae520201
2020-01-14	c6718d0e-976c-4d6c-b0e0-32c770776567
2020-01-15	696c838e-c054-4e9f-a51a-50bf5660f364
2020-01-16	2fa2ab62-f1b0-4036-872f-bcfd9a8686ff
2020-01-17	696c838e-c054-4e9f-a51a-50bf5660f364
2020-01-18	43efce48-94b2-4412-857f-223d45969008
2020-01-19	be676776-8366-4c71-8a35-d58014806eb5
2020-01-20	30a19496-bdaf-461c-abbc-2709ae520201


### 4. Вывести его в формате "<день> <id учителя> <средняя арифметическая оценка>". Оценку можно округлить с точностью до двух знаков после запятой.

In [43]:
for col in day_marks.T.columns:
    wrost_mark = day_marks.T[col].apply(lambda x: float(x)).min()
    bad_teacher = day_marks.T[day_marks.T[col] ==wrost_mark].index[0]
    print ('{}\t{}\t{}'.format(col, bad_teacher, wrost_mark))

2020-01-11	8fe03f08-8581-430c-a590-9888ab36deb3	4.43
2020-01-12	696c838e-c054-4e9f-a51a-50bf5660f364	4.89
2020-01-13	30a19496-bdaf-461c-abbc-2709ae520201	5.0
2020-01-14	c6718d0e-976c-4d6c-b0e0-32c770776567	4.0
2020-01-15	696c838e-c054-4e9f-a51a-50bf5660f364	5.0
2020-01-16	2fa2ab62-f1b0-4036-872f-bcfd9a8686ff	4.0
2020-01-17	696c838e-c054-4e9f-a51a-50bf5660f364	4.5
2020-01-18	43efce48-94b2-4412-857f-223d45969008	4.25
2020-01-19	be676776-8366-4c71-8a35-d58014806eb5	4.5
2020-01-20	30a19496-bdaf-461c-abbc-2709ae520201	4.5
